In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import pickle
from scipy import stats
from sklearn.metrics import classification_report

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install imbalanced_ensemble

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
df = pd.DataFrame(pd.read_csv('/content/drive/MyDrive/Techfest/CBM_WM6undefined_Oct_30.csv'))

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (0,1,4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
df = pd.DataFrame(pd.read_csv('/content/drive/MyDrive/Techfest/CBM_WM6undefined_Oct_30.csv'))

df.drop(df.filter(regex="Unname"),axis=1, inplace=True)
df2=df[['Current','Humidity','Temperature','Flow','Job Temp','Voltage','Defect','Order Operation No']].copy()
df2['Defect'] = df2['Defect'].replace(['No Defect0','No Defect40','No Defect57','No Defect58','No Defect59','No Defect60','No Defect61','No Defect62','No Defect63','No Defect64','No Defect65','No Defect66','No Defect67','No Defect68','No Defect69','No Defect70','No Defect71','No Defect72','No Defect73','No Defect74','No Defect75','No Defect76','No Defect77','No Defect78','No Defect79','No Defect80','No Defect81','No Defect82','No Defect83','No Defect84','No Defect85','No Defect86','No Defect87','No Defect88','No Defect90','No Defect99'],'No Defect')
df2['Defect'] = df2['Defect'].replace(['Porosity71','Porosity75','Porosity76','Porosity80','Porosity86'],'Porosity')
df2['Defect']=df2['Defect'].replace(['Tungsten Inclusion67','Tungsten Inclusion71','Tungsten Inclusion72','Tungsten Inclusion75','Tungsten Inclusion76','Tungsten Inclusion77','Tungsten Inclusion78','Tungsten Inclusion80'],'Tungsten Inclusion')
df2['Defect'] = df2['Defect'].replace(['No defect'],'No Defect')
df2.drop([0],axis=0,inplace=True)
df2.replace([np.inf, -np.inf], np.nan, inplace=True)
df2.fillna(0, inplace=True)
df2.drop(df2[df2['Order Operation No']=='undefined'].index,axis=0,inplace=True)
df2.drop(df2[df2['Order Operation No']=='-'].index,axis=0,inplace=True)
df2.replace('180-', 180, inplace=True)



/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (0,1,4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
from sklearn.model_selection import train_test_split
df2 = df2.sample(frac=1)
X = df2.drop(['Defect'], axis=1)
y = df2['Defect']

x_n_train, x_n_test, y_n_train, y_n_test = train_test_split(
 X, y, test_size=0.3, random_state = 42,stratify = y)

y_n = y_n_test.copy()
y_n.replace(['Porosity','Tungsten Inclusion'],'Defect', inplace=True)

In [ ]:
Y_ed = df2['Defect'].copy()
Y_ed.replace(['Porosity','Tungsten Inclusion'],'Defect', inplace=True)
x_train, x_test, y_train, y_test = train_test_split(
 X, Y_ed, test_size=0.3, random_state = 42, shuffle = False)

In [ ]:
path = '/content/drive/MyDrive/Techfest/Models/brf models'
files = glob.glob(path + '/*.sav')
models = []
for f in files:
  loaded_model = pickle.load(open(f, 'rb'))
  models.append(loaded_model)
x_n_test = x_n_test.astype(float)

yhats = [model.predict(x_n_test) for model in models]
yhats = np.array(yhats)
yhats1 = yhats[:-1,:]
yhats2 = yhats[-1,:]
outcomes, _ = stats.mode(yhats1)
outcomes = np.transpose(outcomes)
print(classification_report(y_n,outcomes,digits =5))

yhats2 = np.transpose(yhats2)
outcomes = np.reshape(outcomes,(-1,1))
yhats2 = np.reshape(yhats2,(-1,1))
for i in range(len(outcomes)):
  if(outcomes[i]=='Defect'):
    outcomes[i] = yhats2[i]

print(classification_report(y_n_test,outcomes,digits =5))

              precision    recall  f1-score   support

      Defect    0.06041   0.99495   0.11390      1781
   No Defect    0.99987   0.72191   0.83845     99107

    accuracy                        0.72673    100888
   macro avg    0.53014   0.85843   0.47618    100888
weighted avg    0.98329   0.72673   0.82566    100888



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                    precision    recall  f1-score   support

         No Defect    0.99987   0.72191   0.83845     99107
          Porosity    0.00000   0.00000   0.00000         0
Tungsten Inclusion    0.06098   0.98933   0.11487      1781

          accuracy                        0.72663    100888
         macro avg    0.35362   0.57041   0.31778    100888
      weighted avg    0.98330   0.72663   0.82568    100888



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
